In [59]:
import pandas as pd

In [29]:
dados_universidade=pd.read_csv('cwurData.csv')
dados_universidade
filtra_us=dados_universidade['country']== 'USA'
filtra_us
dados_uni_us=dados_universidade.loc[filtra_us,:]
dados_uni_us

filtra_ano=dados_uni_us['year']==2015

dados_uni_ano=dados_uni_us.loc[filtra_ano,:]

dados_uni_ano

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1200,1,Harvard University,USA,1,1,1,1,1,1,1,1.0,3,100.00,2015
1201,2,Stanford University,USA,2,9,2,4,5,3,3,4.0,10,98.66,2015
1202,3,Massachusetts Institute of Technology,USA,3,3,11,2,15,2,2,2.0,1,97.54,2015
1205,6,Columbia University,USA,4,13,6,9,13,13,11,12.0,4,96.14,2015
1206,7,"University of California, Berkeley",USA,5,5,21,6,10,4,4,7.0,29,92.25,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100,901,University of Southern Mississippi,USA,225,367,567,218,913,853,812,850.0,676,44.13,2015
2105,906,Oakland University,USA,226,367,567,218,888,810,812,850.0,871,44.13,2015
2112,913,University of North Dakota,USA,227,367,567,218,917,731,812,867.0,606,44.12,2015
2128,929,University of Texas at El Paso,USA,228,367,442,218,910,838,812,906.0,706,44.10,2015


In [35]:
expectativa_vida=pd.read_csv('Life_Expectancy_Data.csv')
filtra_us=expectativa_vida['Country']== 'United States of America'
expectativa_vida_us=expectativa_vida.loc[filtra_us,:]

filtra_ano=expectativa_vida_us['Year']==2015

expectativa_vida_us=expectativa_vida_us.loc[filtra_ano,:]

expectativa_vida_us

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
2794,United States of America,2015,Developed,79.3,13.0,23,NaN,0.0,92.0,188,...,93.0,NaN,95.0,0.1,NaN,NaN,0.8,0.6,NaN,NaN


In [64]:
crimes_armados=pd.read_csv('gun_violence.csv')
crimes_armados

crimes_armados_estados=crimes_armados.loc[:,['state']]
crimes_armados_estados['state'].value_counts(True)
crimes_armados_estados

,state
0,Pennsylvania
1,California
2,Ohio
3,Colorado
4,North Carolina
...,...
239672,Louisiana
239673,Louisiana
239674,Louisiana
239675,Texas


In [57]:
estados=pd.read_csv('states_all.csv')
estados
filtro=estados['YEAR']==2015

estados_2015=estados.loc[filtro,['ENROLL','PRIMARY_KEY']]
estados_2015

,ENROLL,PRIMARY_KEY
1173,734974.0,2015_ALABAMA
1174,130755.0,2015_ALASKA
1175,944978.0,2015_ARIZONA
1176,479682.0,2015_ARKANSAS
1177,6226523.0,2015_CALIFORNIA
1178,872320.0,2015_COLORADO
1179,505366.0,2015_CONNECTICUT
1180,121845.0,2015_DELAWARE
1181,46155.0,2015_DISTRICT_OF_COLUMBIA
1182,2743641.0,2015_FLORIDA


In [68]:
merged = pd.merge(dados_uni_ano, expectativa_vida_us)

#merged2 = pd.merge(merged, crimes_armados_estados, on=['state'], how='left')
#merged3 = pd.merge(merged2, estados_2015, on=['PRIMARY_KEY'], how='left')

TypeError: object of type 'NoneType' has no len()